In [1]:
import numpy as np
import pandas as pd

import re
import tweepy

In [2]:
# Generate your own at https://apps.twitter.com/app
secret = open('secret.txt')
strs = secret.read().split("\n")
CONSUMER_KEY = strs[0]
CONSUMER_SECRET = strs[1]
OAUTH_TOKEN = strs[2] 
OAUTH_TOKEN_SECRET = strs[3]

# connect to twitter
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
api = tweepy.API(auth,wait_on_rate_limit=True)


# batch size depends on Twitter limit, 100 at this time
batch_size = 100

#Some emojis have character length of more than 1
emoji_threshold = 3  

In [69]:
emoji_info = pd.read_csv('emoji_map_1791.csv', sep=',', encoding = 'utf8', engine='c', header = 0)
emoji_info['Unnamed: 0'].head()

emoji_info['Unnamed: 0'][1392]

'😍'

In [74]:
def emoji_statistics(table):
    emoji_counts = {}
    for row in table.iterrows():
        index = (row[0])
        emojis = row[1][2].split(',')
        
        for emoji in emojis:
            if(emoji in emoji_counts.keys()):
                emoji_counts[emoji] += 1
            else:
                emoji_counts[emoji] = 1 
    
    # Stats about the dataset
    print('# different emojis: ', len(emoji_counts.keys()))
    vals = list(emoji_counts.values())
    print('min emoji count: ', min(vals))
    print('max emoji count: ', max(vals))
    print('mean emoji count: ', np.mean(vals))
    print('median emoji count: ', np.median(vals))
    print('Standard Deviation emoji count: ', np.std(vals))
    
    # Stats about the most used emojis
    {value : key for key, value in emoji_counts.items()}
    
    sorted_emoji_counts = sorted(emoji_counts.items(), key= lambda kv: kv[1], reverse=True)
    
    print('Stats about the most used emojis (descending): \n')
    for key, value in sorted_emoji_counts[20]:
        emoji = emoji_info['Unnamed: 0'][int(key)]
        print('Emoji: ', emoji)
        print('Count: ', value)
        print()
    
    return sorted_emoji_counts

In [5]:
files = ['img_train_plaintext.txt', 'img_test_plaintext.txt', 'img_valid_plaintext.txt']
file = files[0]

# Read Data from the file
data = pd.read_csv(file, sep='\t', encoding = 'utf8', engine='c', header = 0)
data.head()

,id,imgid,annotations
0,744014442837454848,http://pbs.twimg.com/media/ClNFMNSWgAI7qq0.jpg,"47,883"
1,742240402288345088,http://pbs.twimg.com/media/CkzuE33WsAA2cUB.jpg,229
2,742427564535353349,http://pbs.twimg.com/media/Ck2h7lzXIAA0Aqw.jpg,658
3,747888709375229952,http://pbs.twimg.com/media/CmEIe-_VYAADvg0.jpg,"1108,1108"
4,747199027649871872,http://pbs.twimg.com/media/Cl6UJF1XIAADDAJ.jpg,1466


In [75]:
emoji_statistics(data)

# different emojis:  1254
min emoji count:  1
max emoji count:  81864
mean emoji count:  1356.8141945773525
median emoji count:  123.0
Standard Deviation emoji count:  4919.49807459935
Stats about the most used emojis (descending): 



ValueError: too many values to unpack (expected 2)

In [35]:
#  Remove tweets with more than 1 emoji

dataToDrop = []
for row in data.iterrows():
    index = (row[0])
    emoji_count = len(row[1][2].split(','))
    if(emoji_count > 1):
        dataToDrop.append(index)
new_data = data.drop(data.index[dataToDrop])

In [36]:

print(len(data))
print(len(new_data))

916864
589573


In [72]:
sorted_counts = emoji_statistics(new_data)

Emoji:  😍
Count:  41176
Emoji:  ❤
Count:  36198
Emoji:  😂
Count:  35501
Emoji:  💕
Count:  18298
Emoji:  😊
Count:  15696
Emoji:  ✨
Count:  10253
Emoji:  😘
Count:  10002
Emoji:  😭
Count:  9832
Emoji:  💖
Count:  9064
Emoji:  🔥
Count:  8985
Emoji:  💙
Count:  8968
Emoji:  😎
Count:  8246
Emoji:  💜
Count:  6733
Emoji:  💗
Count:  6627
Emoji:  😁
Count:  6327
Emoji:  😉
Count:  6114
Emoji:  💋
Count:  6027
Emoji:  🤔
Count:  5835
Emoji:  👀
Count:  5829
Emoji:  🔴
Count:  5796


In [6]:
def locate_emoji(emoji_pattern, text: str):
    emoji = ''.join(emoji_pattern.findall(text))
    try:
        index = text.index(emoji)
    except:
        index = - emoji_threshold
    return emoji, index

In [13]:
def get_tweets(twapi, data):
    '''
    Fetches content for tweet IDs in a file using bulk request method,
    which vastly reduces number of HTTPS requests compared to above;
    however, it does not warn about IDs that yield no tweet.
    `twapi`: Initialized, authorized API object from Tweepy
    '''
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u'\U00010000-\U0010ffff'
        u"\u200d"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\u3030"
        u"\ufe0f"
        "]+",
        flags=re.UNICODE)

    tweet_ids = data.id.values.tolist()

    all_tweets = []
    emojis = []
    i = 0  #for debug
    
    # process list of ids until it's empty
    while len(tweet_ids) > 0:
        if len(tweet_ids) < batch_size:
            tweets = twapi.statuses_lookup(
                id_=tweet_ids, include_entities=False, trim_user=True)
            tweet_ids = []
        else:
            tweets = twapi.statuses_lookup(
                id_=tweet_ids[:batch_size],
                include_entities=False,
                trim_user=True)
            tweet_ids = tweet_ids[batch_size:]

        for tweet in tweets:
            
            #removes the link of the tweet
            text = re.sub(r'http\S+', '', tweet.text).strip(' ')            
            text = re.sub(r'\ART @\S+','', text).strip(' ')
            
            #Remove tweets where emoji is not at the end
            emoji, index = locate_emoji(emoji_pattern, text)
            
            if index >= len(text) - emoji_threshold:
                #removes the emojis from the text
                text = emoji_pattern.sub(r'', text).strip(' ')

                #then appends the tweet and emoji to our final dataset
                all_tweets.append(np.array([text]))
                emojis.append(emoji)

        i += 1
        #if i == 20:
        #    break
            
    features = all_tweets
    labels = emojis
    #features = np.array(all_tweets)
    #labels = np.array(labels)
    return features, labels

In [37]:
X, y = get_tweets(api, new_data)

RateLimitError: [{'message': 'Rate limit exceeded', 'code': 88}]

In [38]:
print(len(X))

1580


In [10]:
y

['😒',
 '💔',
 '🙏',
 '👌',
 '❤️',
 '💙',
 '💕',
 '😹',
 '🎶',
 '👊',
 '🐠',
 '😘',
 '💖',
 '🚣🏽',
 '🌸',
 '🎥',
 '💟',
 '😍',
 '💖',
 '😊',
 '😁',
 '🐐',
 '🏉',
 '😢',
 '😏',
 '💕',
 '😱',
 '🙄',
 '👌',
 '😱',
 '😍',
 '😁',
 '💛',
 '😍',
 '💖',
 '😂',
 '❤️',
 '❤️',
 '👍',
 '✌️',
 '😂',
 '💪',
 '😋',
 '🌍',
 '😛',
 '👌',
 '🌅',
 '🌚',
 '🇨🇩',
 '👏🏻',
 '😂',
 '❤️',
 '👎',
 '❤️',
 '💯',
 '🌞',
 '🎉',
 '☺️',
 '😛',
 '🇺🇸',
 '♠️',
 '🤐',
 '💀',
 '❤️',
 '😕',
 '💜',
 '👶🏼',
 '😌',
 '😨',
 '🔥',
 '🐴',
 '❤️',
 '💚',
 '🍸',
 '😌',
 '💕',
 '💞',
 '😍',
 '🍐',
 '👍',
 '❤️',
 '😍',
 '🤔',
 '🌻',
 '😕',
 '❗️',
 '👀',
 '😊',
 '😄',
 '❤️',
 '😉',
 '🎥',
 '✨',
 '🔨',
 '😎',
 '🐳',
 '💣',
 '😒',
 '👋',
 '💕',
 '😍',
 '😹',
 '😍',
 '😊',
 '😊',
 '🐾',
 '👋🏼',
 '😁',
 '✨',
 '💙',
 '😭',
 '🙌🏾',
 '🙄',
 '😜',
 '👽',
 '💕',
 '😜',
 '💖',
 '🎆',
 '😂',
 '😃',
 '😉',
 '✅',
 '😉',
 '😂',
 '💟',
 '💓',
 '💋',
 '👉',
 '😊',
 '🍾',
 '🏆',
 '🐸',
 '😍',
 '👑',
 '💗',
 '❤️',
 '💘',
 '💗',
 '💙',
 '😊',
 '😞',
 '🎶',
 '👑',
 '❤️',
 '😍',
 '🌴',
 '💛',
 '🔊',
 '👽',
 '😝',
 '😇',
 '❤️',
 '💕',
 '😍',
 '🐾',
 '💺',
 '😸',
 '😨',
 '💕',
 '🙏🏼',
 '😜',
 '💓'

In [12]:
len(set(y))

340